In [1]:
from collections import deque
from typing import Any, Optional

In [2]:
# Implement (code) a Key value store with transactions.

# Write a Fully funcitonal code in 25-30 min in interview with test cases

# Set
# Get
# Delete are methods in Key value store

# for transactions
# Begin
# Commit
# Rollback

In [3]:
class DB:
    def __init__(self):
        self.current = {}
        self.stack = deque()
        self.open_transactions = 0
        
    def get(self, value: Any) -> Optional[Any]:
        return self.current.get(value)
    
    def set(self, key: Any, value: Any) -> None:
        if self.open_transactions == 0:
            self.current[key] = value
            return

        self.stack.append({key: value})
        
    def delete(self, key: int) -> None:
        self.current.pop(key)
        
    def commit(self):
        if self.open_transactions == 0:
            return

        changes = {}
        while len(self.stack) > 0:
            curr = self.stack.pop()

            if curr == "X_BLOCK":
                break
            # for any/all transactions add to temporary change store
            changes.update(curr)

        # finally, update main data store
        self.current.update(changes)

        # reduce transaction count
        self._update_transaction_count(-1)
        
        
    def begin(self):
        self._update_transaction_count(1)
        self.stack.append("X_BLOCK")
        self.current = dict(self.current)
        
    def rollback(self):
        if self.open_transactions == 0:
            return

        while len(self.stack) != 0:
            curr = self.stack.pop()
            if curr == "X_BLOCK":
                break

        self._update_transaction_count(-1)
        
    def _update_transaction_count(self, val: int) -> None:
        self.open_transactions += val
        self.open_transactions = abs(self.open_transactions)

In [3]:
class DB:
    
    def __init__(self):
        self.current = {}
        self.stack = deque()
        self.open_transactions = 0
        
    def get(self, value: Any) -> Optional[Any]:
        return self.current.get(value)
    
    def set(self, key: Any, value: Any) -> None:
        if self.open_transactions==0:
            self.current[key]=value
            return
        self.stack.append({key:value})
        
    def delete(self, key: int) -> None:
        self.current.pop(key)
        
    def begin(self):
        self._update_transaction_count(1)
        self.stack.append("X")
        self.current = dict(self.current)
        
    def _update_transaction_count(self, val: int) -> None:
        self.open_transactions+=val
        self.open_transactions = abs(self.open_transactions)
        
    def commit(self):
        if self.open_transactions==0:
            return
        
        changes = {}
        while len(self.stack)>0:
            current = self.stack.pop()
            if current=="X":
                break
            changes.update(current)
        
        self.current.update(changes)
        self._update_transaction_count(-1)
        
    def rollback(self):
        if self.open_transactions==0:
            return
        
        while len(self.stack)!=0:
            curr = self.stack.pop()
            if curr=="X":
                break
                
        self._update_transaction_count(-1)

In [4]:
kv = DB()
kv.set(1, 3)
kv.set(2, 4)

# begin transaction
kv.begin()
kv.set(3, 5)
kv.set(6, 6)

# nested transaction
kv.begin()
kv.set(11, 11)

assert kv.get(1) == 3
assert kv.get(2) == 4
assert kv.get(3) is None
assert kv.get(6) is None
assert kv.get(11) is None

kv.commit()

assert kv.get(11) == 11

# commit
kv.commit()

assert kv.get(1) == 3
assert kv.get(2) == 4
assert kv.get(3) == 5
assert kv.get(6) == 6

# begin transaction
kv.begin()
kv.set(7, 7)
kv.set(8, 8)

assert kv.get(7) is None
assert kv.get(8) is None

# nested transaction
kv.begin()
kv.set(13, 13)

# rollback 13
kv.rollback()

assert kv.get(13) is None
assert kv.get(7) is None
assert kv.get(8) is None

# rollback test
kv.rollback()

# test rollback with no open transactions
# should do nothing
kv.rollback()

# test commit with no active transactions
# should do nothing
kv.commit()

assert kv.get(1) == 3
assert kv.get(2) == 4
assert kv.get(3) == 5
assert kv.get(6) == 6
assert kv.get(7) is None
assert kv.get(8) is None

# test delete
kv.delete(1)

assert kv.get(1) is None